<a href="https://colab.research.google.com/github/Siddhartha0310/nlp_practicals/blob/main/kindle_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Kindle reveiw sentiment analysis

Data set from https://jmcauley.ucsd.edu/data/amazon/

Dataset has the follwing the xolumns
reviewerID - ID of the reviewer, e.g. A2SUAM1J3GNN3B
asin - ID of the product, e.g. 0000013714
reviewerName - name of the reviewer
helpful - helpfulness rating of the review, e.g. 2/3
reviewText - text of the review
overall - rating of the product
summary - summary of the review
unixReviewTime - time of the review (unix time)
reviewTime - time of the review (raw)

Best practices the will followed:-
preprocessing
Train test split
BOW,Tfidf,word2vec
train ml alogorithms

In [205]:
import pandas as pd
data=pd.read_csv("/content/all_kindle_review.csv")
data.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [206]:
data=data[['reviewText','rating']]
data

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4
...,...,...
11995,Valentine cupid is a vampire- Jena and Ian ano...,4
11996,I have read all seven books in this series. Ap...,5
11997,This book really just wasn't my cuppa. The si...,3
11998,"tried to use it to charge my kindle, it didn't...",1


In [207]:
data.isna().sum()

,0
reviewText,0
rating,0


In [208]:
data['rating'].unique()

array([3, 5, 4, 2, 1])

In [209]:
data['rating'].value_counts()

,count
rating,
5,3000
4,3000
3,2000
2,2000
1,2000


In [210]:
#preprocessing and cleaning
data['rating']=data['rating'].apply(lambda x: 0 if x<3 else 1 )

In [211]:
data['rating'].value_counts()

,count
rating,
1,8000
0,4000


In [212]:
## lower all the cases
data['reviewText']=data['reviewText'].str.lower()
data.head()

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [213]:
!pip install beautifulsoup4 lxml

In [214]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [215]:
import re
from nltk.corpus import stopwords
from bs4 import BeautifulSoup

data['reviewText'] = data['reviewText'].apply(lambda x: re.sub(r'[^a-z0-9\s\-]', '', x))
stop_words = set(stopwords.words('english'))  # Use set for faster lookup
data['reviewText'] = data['reviewText'].apply(
    lambda x: " ".join(y for y in x.split() if y and y not in stop_words)
)
data['reviewText'] = data['reviewText'].apply(
    lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '', str(x))
)
data['reviewText'] = data['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())
data['reviewText'] = data['reviewText'].apply(lambda x: " ".join(x.split()))

In [216]:
data.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


In [217]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemtizer=WordNetLemmatizer()

def lemmatixe(text):
  return " ".join([lemtizer.lemmatize(word) for word in text.split()])


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [218]:
data['reviewText']=data['reviewText'].apply(lambda x: lemmatixe(x))

In [219]:
data.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


In [220]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['reviewText'], data['rating'], test_size=0.20, random_state=42)

In [221]:
from sklearn.feature_extraction.text import CountVectorizer
bow=CountVectorizer()
X_train_bow = bow.fit_transform(X_train).toarray()
X_test_bow = bow.transform(X_test).toarray()


In [222]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train).toarray()
X_test_tfidf = tfidf.transform(X_test).toarray()


In [223]:
from sklearn.naive_bayes import GaussianNB
nb_model_bow = GaussianNB().fit(X_train_bow, y_train)
nb_model_tfidf = GaussianNB().fit(X_train_tfidf, y_train)



In [224]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
y_pred_bow =nb_model_bow.predict(X_test_bow)
print('Bag of words confusion matrix:-',confusion_matrix(y_test,y_pred_bow))
print('Bag of words accuracy:-',accuracy_score(y_test,y_pred_bow))
print('Bag of words classifaction report:-',classification_report(y_test,y_pred_bow))

y_pred_tdif =nb_model_tfidf.predict(X_test_tfidf)
print('term frequency-inverse document frequency',confusion_matrix(y_test,y_pred_tdif))
print('term frequency-inverse document frequency',accuracy_score(y_test,y_pred_tdif))
print('term frequency-inverse document frequency',classification_report(y_test,y_pred_tdif))

Bag of words confusion matrix:- [[499 304]
 [717 880]]
Bag of words accuracy:- 0.5745833333333333
Bag of words classifaction report:-               precision    recall  f1-score   support

           0       0.41      0.62      0.49       803
           1       0.74      0.55      0.63      1597

    accuracy                           0.57      2400
   macro avg       0.58      0.59      0.56      2400
weighted avg       0.63      0.57      0.59      2400

term frequency-inverse document frequency [[488 315]
 [696 901]]
term frequency-inverse document frequency 0.57875
term frequency-inverse document frequency               precision    recall  f1-score   support

           0       0.41      0.61      0.49       803
           1       0.74      0.56      0.64      1597

    accuracy                           0.58      2400
   macro avg       0.58      0.59      0.57      2400
weighted avg       0.63      0.58      0.59      2400



Since precision is not good applying word2Vec

In [225]:
!pip install gensim
import nltk
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [227]:
import gensim
from gensim.models import Word2Vec,KeyedVectors
from nltk.tokenize import word_tokenize

data['reviewText']=data['reviewText'].apply(lambda x: word_tokenize(x))



In [228]:
model=gensim.models.Word2Vec(data['reviewText'],vector_size=100)

In [229]:
import numpy as np
def averageWord2Vec(doc):
  return np.mean([model.wv[word] for word in doc if word in model.wv],axis=0)

In [230]:
!pip install tqdm
from tqdm import tqdm

In [231]:
doc=[]
for i in tqdm(range(data['reviewText'].size)):
  doc.append(averageWord2Vec(data['reviewText'][i]))

100%|██████████| 12000/12000 [00:04<00:00, 2520.46it/s]


In [232]:
doc_new =np.array(doc)
doc_new

array([[-0.06211099,  0.29515418,  0.14177346, ..., -0.37245804,
         0.07631791, -0.1914524 ],
       [-0.01157395,  0.1850049 ,  0.01588978, ..., -0.21356373,
         0.06947619, -0.56758076],
       [-0.11719135,  0.23035468,  0.01230501, ..., -0.3066569 ,
         0.04385082, -0.2336762 ],
       ...,
       [-0.10438087,  0.3003359 ,  0.11847988, ..., -0.19961116,
        -0.01206352, -0.5020588 ],
       [ 0.22948132,  0.28704375,  0.1169333 , ..., -0.63178545,
         0.16470245, -0.07901491],
       [-0.05508758,  0.20279314,  0.14048362, ..., -0.18303797,
         0.02441156, -0.32540745]], dtype=float32)

In [233]:
from sklearn.model_selection import train_test_split
X_train_vec, X_test_vec, y_train_vec, y_test_vec = train_test_split(doc_new, data['rating'], test_size=0.20, random_state=42)

In [234]:
from sklearn.naive_bayes import GaussianNB
nb_model_bow = GaussianNB().fit(X_train_vec, y_train)


In [236]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
y_pred_vec =nb_model_bow.predict(X_test_vec)
print('woerd2vec confusion matrix:-',confusion_matrix(y_test,y_pred_vec))
print('woerd2vec accuracy score:-',accuracy_score(y_test,y_pred_vec))
print('woerd2vec claasifcation report:-',classification_report(y_test,y_pred_vec))


woerd2vec confusion matrix:- [[ 592  211]
 [ 449 1148]]
woerd2vec accuracy score:- 0.725
woerd2vec claasifcation report:-               precision    recall  f1-score   support

           0       0.57      0.74      0.64       803
           1       0.84      0.72      0.78      1597

    accuracy                           0.72      2400
   macro avg       0.71      0.73      0.71      2400
weighted avg       0.75      0.72      0.73      2400

